Farmer problem (here-and-now) policy

Mauricio Tombini and Victor M. Zavala 2016 

University of Wisconsin-Madison  

In [6]:
using JuMP 
using Ipopt 

In [9]:
# Model parameters

NS = 5;                    # number of scenarios
S = collect([1:NS]);       # scenario set
P = collect([1:3]);        # set of crops (1=wheat,2=corn,3=beets)

# Data 
prcost = zeros(3)          # production (planting) cost
prcost[1] = 150;
prcost[2] = 230;
prcost[3] = 260;

pcost = zeros(3)           # purchase cost
pcost[1] = 238;
pcost[2] = 210;
pcost[3] = 0;

scost = zeros(3)           # sales cost
scost[1] = 170;
scost[2] = 150;
scost[3] = 36;

demand = zeros(3)          # demand
demand[1] = 200;
demand[2] = 240;
demand[3] = 0;

# assign random data

yield = zeros(length(S),3) # yields
yield[S,1] = 2.5;
yield[S,2] = 3.0;

yield[1,3] = 10;
yield[2,3] = 15;
yield[3,3] = 20;
yield[4,3] = 25;
yield[5,3] = 30

30

In [10]:
# Model 

m = Model(solver=IpoptSolver(print_level=0))

@defVar(m, x[S,P] >= 0)    # acres devoted to crops
@defVar(m, y[S,P] >= 0)    # crops purchase
@defVar(m, w[S,P] >= 0)    # crops sold;
@defVar(m, cost[s in S])   # per scenario cost

@addConstraint(m, varcost[s in S], cost[s] == sum{prcost[j]*x[s,j] + pcost[j]*y[s,j] - scost[j]*w[s,j], j in P}) 
@addConstraint(m, cap[s in S], sum{x[s,j], j in P} <= 500)
@addConstraint(m, bal[s in S,j in P], yield[s,j]*x[s,j]+y[s,j]-w[s,j] >= demand[j]) 
@addConstraint(m, sellb[s in S], w[s,3] <= 6000)
@addConstraint(m, buyb[s in S], y[s,3] <= 0)
@addConstraint(m, nonant[s in S,j in P], x[1,j] == x[s,j])

@setObjective(m, Min, (1/NS)*sum{cost[s], s in S})

solve(m)

:Optimal

In [11]:
# Results

println(getValue(x))
println("")
println(getValue(y))
println("")
println(getValue(w))
println("")
println(getValue(cost))
println("")
println("obj ",getObjectiveValue(m))

x: 2 dimensions, 15 entries:
 [1,1] = 220.00000380066524
 [1,2] = 79.9999992000006
 [1,3] = 200.00000199933396
 [2,1] = 220.00000380066524
 [2,2] = 79.9999992000006
 [2,3] = 200.00000199933396
 [3,1] = 220.00000380066524
 [3,2] = 79.9999992000006
 [3,3] = 200.00000199933396
 [4,1] = 220.00000380066524
 [4,2] = 79.99999920000062
 [4,3] = 200.00000199933396
 [5,1] = 220.00000380066524
 [5,2] = 79.99999920000062
 [5,3] = 200.00000199933396

y: 2 dimensions, 15 entries:
 [1,1] = 0.0
 [1,2] = 0.0
 [1,3] = 9.99861130401085e-9
 [2,1] = 0.0
 [2,2] = 0.0
 [2,3] = 9.998611304001737e-9
 [3,1] = 0.0
 [3,2] = 0.0
 [3,3] = 9.998611304009665e-9
 [4,1] = 0.0
 [4,2] = 0.0
 [4,3] = 9.998614010973276e-9
 [5,1] = 0.0
 [5,2] = 0.0
 [5,3] = 9.990331945867265e-9

w: 2 dimensions, 15 entries:
 [1,1] = 350.0000114916636
 [1,2] = 0.0
 [1,3] = 2000.0000200133368
 [2,1] = 350.0000114916636
 [2,2] = 0.0
 [2,3] = 3000.0000300100064
 [3,1] = 350.0000114916636
 [3,2] = 0.0
 [3,3] = 4000.0000400066765
 [4,1] = 350.000